In [27]:
import os
import numpy as np
import pandas as pd
from copy import deepcopy
import difflib


## Loading CSVs

In [28]:
ml_xlsx_file_sample = reviewers_xlsx_file_sample = None
ml_xlsx_file = reviewers_xlsx_file = None

In [29]:
REPO_PATH = '/home/celo/Desktop/git-repo-slr/SLR-Automated_selection_of_studies'
RESULTS_PATH = '/home/celo/Desktop/data-results'

ml_xlsx_file_path = 'output/svm-only/kfold-anovaf/zenodo-table.xlsx'
reviewers_xlsx_file_path = 'open-repo-papers-zenodo/full-table.xlsx'
# reviewers_xlsx_file = '/home/mcostalonga/new-home/thesis/open-repo-papers-zenodo/full-table.xlsx' # Attempt1 | Sheet1

# ml_xlsx_file_sample = '/home/mcostalonga/new-home/thesis/git-repo-slr/SLR-Automated_selection_of_studies/output/svm-only/kfold-anovaf/ml_test.xlsx'
# reviewers_xlsx_file_sample = '/home/mcostalonga/new-home/thesis/git-repo-slr/SLR-Automated_selection_of_studies/output/svm-only/kfold-anovaf/reviewers_test.xlsx'

# ml_xlsx_file_sample = '/home/mcostalonga/new-home/thesis/git-repo-slr/SLR-Automated_selection_of_studies/output/svm-only/kfold-anovaf/ml_test_big_sample.xlsx'
# reviewers_xlsx_file_sample = '/home/mcostalonga/new-home/thesis/git-repo-slr/SLR-Automated_selection_of_studies/output/svm-only/kfold-anovaf/reviewers_test_big_sample.xlsx'

ml_xlsx_file = os.path.join(REPO_PATH, ml_xlsx_file_path)
reviewers_xlsx_file = os.path.join(RESULTS_PATH, reviewers_xlsx_file_path)


In [30]:
# Sample files to test notebook
if ml_xlsx_file_sample and reviewers_xlsx_file_sample:
    ml_pd = pd.read_excel(ml_xlsx_file_sample, sheet_name='Probabilities', engine='openpyxl')
    desired_columns = ["Titles", "Claes", "Marcos", "Katia"]
    reviewers_pd = pd.read_excel(reviewers_xlsx_file_sample, sheet_name='Probabilities', engine='openpyxl', usecols=desired_columns)

# Real complete files
elif ml_xlsx_file and reviewers_xlsx_file:
    ml_pd = pd.read_excel(ml_xlsx_file, sheet_name='Probabilities', engine='openpyxl')
    desired_columns = ["Titles", "Claes", "Marcos", "Katia"]
    reviewers_pd = pd.read_excel(reviewers_xlsx_file, sheet_name='Sheet1', engine='openpyxl', usecols=desired_columns)

else:
    print('[ERROR] Inform valid CSVs files to read')
    raise Exception

ml_pd = ml_pd.iloc[: ,1:]  # Only execute this once to remove the first unused column


In [31]:
# OBS: Titles with typos written on Testing sets 
# # Apply this just to make sure they don't go unoticed

# ml_titles_with_typos = {
#     'A COLABORA{\c{C}}{\~A}O UNIVERSIDADE-EMPRESA NO CONTEXTO DA IND{\'U}STRIA 4.0': 'A COLABORAÇÃO UNIVERSIDADE-EMPRESA NO CONTEXTO DA INDÚSTRIA 4.0'
# }
ml_pd['Titles'] = ml_pd['Titles'].str.replace(r'{\\c{C}}', 'Ç')
ml_pd['Titles'] = ml_pd['Titles'].str.replace(r'{\\~A}', 'Ã')
ml_pd['Titles'] = ml_pd['Titles'].str.replace(r'{\'U}', 'Ú')


/tmp/ipykernel_44107/3175091779.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  ml_pd['Titles'] = ml_pd['Titles'].str.replace(r'{\\c{C}}', 'Ç')
/tmp/ipykernel_44107/3175091779.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  ml_pd['Titles'] = ml_pd['Titles'].str.replace(r'{\\~A}', 'Ã')
/tmp/ipykernel_44107/3175091779.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  ml_pd['Titles'] = ml_pd['Titles'].str.replace(r'{\'U}', 'Ú')


In [32]:
# ML's CSV summary
print('[ML] Num of rows:', len(ml_pd))
ml_pd.head(15)


[ML] Num of rows: 581


,Titles,Was Selected?,SVM_proba
0,Operations Technology and Organizational Struc...,0,0.980527
1,Design science as nested problem solving,0,0.513517
2,A Large-Scale Empirical Study of Practitioners...,0,0.289893
3,Towards an Approach Matching CMD and DSR to Im...,0,0.983313
4,A PROPOSAL FOR USING DESIGN SCIENCE IN EDUCATI...,0,0.576047
5,University-industry collaboration and open sou...,0,0.567045
6,A bibliometric analysis of the Turkish softwar...,0,0.653437
7,An Improved Indoor Positioning Method Based on...,0,0.882294
8,Industry-academia collaboration in software te...,0,1.000000
9,Measuring Software Reliability: A Trend Using ...,0,0.128433


In [33]:
new_reviewers_pd = deepcopy(reviewers_pd)

string_to_numeric = {
    'Assessed, citing SLR': 0,
    'Assessed original SLR': 0,
    'Assessed Batch 1': 0,
    'Assessed': 0,
    'PhD thesis': 0,
    'Bachelor thesis': 0,
    'Licentiate thesis': 0,
    'Not English': 0,
    'Original SLR': 0,
    'Book': 0,
    'Book (chapter in own book)': 0,
    'SMS': 0,
    'SLR': 0,
    'Keynote': 0,
    'See line 55': 0,
    'See line 63': 0,
    'See line 66': 0,
    'See line 108': 0,
    'See line 178': 0,
    'See line 205': 0,
    'Literature review': 0,
    'Review article': 0,
    'Panel summary': 0,
    'Included Assessed original SLR': 2,
    'Included Assessed, citing SLR': 2,
    'Included Assessed': 2
}
for column in ['Claes', 'Marcos', 'Katia']:
    new_reviewers_pd[column] = new_reviewers_pd[column].replace(string_to_numeric)
# OBS: Line 7 should be all 0's after this change

# Reviewers' CSV summary
print('[Reviewers] Num of rows:', len(new_reviewers_pd))
new_reviewers_pd.head(10)

[Reviewers] Num of rows: 540


,Titles,Claes,Marcos,Katia
0,Structuring automotive product lines and featu...,0,0,0
1,Using scrum in outsourced government projects:...,0,0,0
2,Interventions for software security: creating ...,0,0,0
3,Sustainability analysis and ease of learning i...,0,0,0
4,Light-touch interventions to improve software ...,0,0,0
5,Academic and industrial software testing confe...,0,0,0
6,Applying data analytics towards optimized issu...,0,0,0
7,Case studies of industry-academia research col...,0,0,0
8,Wide Band Patch Antenna Structures for Cogniti...,0,0,0
9,Interventions for long‐term software security:...,0,0,0


In [34]:
# Reviewers' CSV summary
print('[Reviewers] Num of rows:', len(reviewers_pd))
reviewers_pd.head(10)

[Reviewers] Num of rows: 540


,Titles,Claes,Marcos,Katia
0,Structuring automotive product lines and featu...,0,0,0
1,Using scrum in outsourced government projects:...,0,0,0
2,Interventions for software security: creating ...,0,0,0
3,Sustainability analysis and ease of learning i...,0,0,0
4,Light-touch interventions to improve software ...,0,0,0
5,Academic and industrial software testing confe...,0,0,0
6,Applying data analytics towards optimized issu...,0,0,0
7,Case studies of industry-academia research col...,"Assessed, citing SLR","Assessed, citing SLR","Assessed, citing SLR"
8,Wide Band Patch Antenna Structures for Cogniti...,0,0,0
9,Interventions for long‐term software security:...,0,0,0


In [35]:
# sorted_df = reviewers_pd.sort_values(by='Titles', ascending=True)
# sorted_df.head()
reviewers_pd = new_reviewers_pd
reviewers_pd.head(10)

,Titles,Claes,Marcos,Katia
0,Structuring automotive product lines and featu...,0,0,0
1,Using scrum in outsourced government projects:...,0,0,0
2,Interventions for software security: creating ...,0,0,0
3,Sustainability analysis and ease of learning i...,0,0,0
4,Light-touch interventions to improve software ...,0,0,0
5,Academic and industrial software testing confe...,0,0,0
6,Applying data analytics towards optimized issu...,0,0,0
7,Case studies of industry-academia research col...,0,0,0
8,Wide Band Patch Antenna Structures for Cogniti...,0,0,0
9,Interventions for long‐term software security:...,0,0,0


#


#



# RETHINK OF THIS SOLUTION START

### Search for each title from ML in the Reviewers df to find closest match by DiffLib
##### Find how many matches can be found by this method

In [42]:
# # matched_df = merged_dataframe.dropna(subset=merged_dataframe.columns[merged_dataframe.isna().any()].tolist())
# i = 0
# # similarity_treshold = 85  
# # similarity_treshold = 75  
# similarity_treshold = 50   # 567 matches (only missed 21... REMEMBER THAT ML CSV IS BIGGER THAN REVIEWERS'...)
# difflib_treshold = similarity_treshold/100

# # Format both dataframes to lower case so is easier to match
# formated_ml_df = deepcopy(ml_pd)
# formated_ml_df['Titles_Formated'] = formated_ml_df['Titles'].str.lower()
# formated_ml_df['_match_index'] = np.nan  # Add new column and set all rows to be NaN

# formated_reviewers_df = deepcopy(reviewers_pd)
# formated_reviewers_df['Titles_Formated'] = formated_reviewers_df['Titles'].str.lower()
# formated_reviewers_df['_match_index'] = np.nan  # Add new column and set all rows to be NaN

# difflib_match_count = 0
# matches = dict()

# while(i < len(formated_ml_df)-1):
# # while(i < 3):
#     curr_ml_title = formated_ml_df.loc[i]['Titles_Formated']

#     # Match by difflib
#     difflib_match = difflib.get_close_matches(curr_ml_title, formated_reviewers_df['Titles_Formated'], n=1, cutoff=difflib_treshold)
   
#     if difflib_match:
#             # print('DIFF LIB:\n\t ml_title: {}\n\tmatch: {}'.format(curr_ml_title, difflib_match))
#             difflib_match_count+=1

#             # # Use boolean indexing to filter rows where 'Titles' column is equal to 'string_value'
#             matched_row_index = formated_reviewers_df[formated_reviewers_df['Titles_Formated'] == difflib_match[0]].index[0]

#             # Update the column '_match_index' on both dataframes to use to merge them later
#             formated_reviewers_df.at[matched_row_index, '_match_index'] =  difflib_match_count
#             formated_ml_df.at[i, '_match_index'] =  difflib_match_count


#             if difflib_match_count not in matches:
#                   matches[difflib_match_count] = { 'ml_title': curr_ml_title, 'reviewer_match': difflib_match[0] }

#             # print(formated_ml_df.loc[i].head())
#             # formated_ml_df.loc[i].head()
#             # print(formated_reviewers_df.loc[matched_row_index])
#             # formated_reviewers_df.loc[matched_row_index].head()
#             # print(matched_row_reviewers['Titles'], matched_row_reviewers['Claes'], matched_row_reviewers['Marcos'], matched_row_reviewers['Katia'])


#             # TODO: Change the original DataFrame so ML already have a value for each reviewer for matches found
#             # TODO: Need to add three new columns for each reviewer and complete with NaN initially...
#             # TODO: Then, will apply The Fuzz only to the rows that weren't matched...
            

#             # # Copy values from curr_row into next_row for the second and third columns
#             # similar_matches_df.iloc[i+1, 4:7] = similar_matches_df.iloc[i, 4:7]

#             # # Copy values from next_row into curr_row for the Reviewers columns
#             # similar_matches_df.iloc[i, 2:4] = similar_matches_df.iloc[i+1, 2:4]

#             # # Copy values from next_row into curr_row for the Reviewers columns
#             # similar_matches_df.iloc[i, 4:7] = similar_matches_df.iloc[i+1, 4:7]

#             # # Copy values from curr_row into next_row for the second and third columns
#             # similar_matches_df.iloc[i+1, 2:4] = similar_matches_df.iloc[i, 2:4]


#             # matches[curr_ml_title] = difflib_match[0]
#     i+=1

# print('Difflib matches:', difflib_match_count)
# # pd.DataFrame.from_dict(matches).head()


Difflib matches: 567


In [43]:
# # Merge ML's and Reviewers' CSVs into one, by matching Titles

# # formated_reviewers_pd


# # tmp_df = pd.merge(formated_ml_df, formated_reviewers_df, on='_match_index', how='inner')
# tmp_df = pd.merge(formated_ml_df, formated_reviewers_df.drop(['Titles', 'Titles_Formated'], axis=1), on='_match_index', how='inner')

# # tmp_df['_match_index'].fillna(placeholder, inplace=True)
# # tmp_df = tmp_df.dropna(subset=['_match_index'])
# tmp_df = tmp_df.sort_values(by='_match_index', ascending=False)



# print(len(tmp_df))
# tmp_df.head(10)
# # Total of 632, but removing NaN is 537... HOW ??????
# # Maybe titles from REV that were not found/matched on ML... ??

632


,Titles,Was Selected?,SVM_proba,Titles_Formated,_match_index,Claes,Marcos,Katia
631,A Taxonomy of Information Attributes for Test ...,0,0.704314,a taxonomy of information attributes for test ...,567.0,0,0,0
630,Organizing Empirical Studies as Learning Itera...,0,0.981023,organizing empirical studies as learning itera...,566.0,0,0,0
629,The use of Design Science and Agile Methodolog...,0,0.868558,the use of design science and agile methodolog...,565.0,0,0,0
628,Machine Learning for Software Engineering: A T...,0,0.717700,machine learning for software engineering: a t...,564.0,0,0,0
627,Industry?Academia Research Collaboration and K...,1,1.000000,industry?academia research collaboration and k...,563.0,2,2,2
626,Conceptual modeling to support system-level de...,0,0.900588,conceptual modeling to support system-level de...,562.0,0,0,0
625,International technology transfer as an effect...,0,0.823900,international technology transfer as an effect...,561.0,0,0,0
624,Exploring ML testing in practice: lessons lear...,0,0.638226,exploring ml testing in practice: lessons lear...,560.0,0,0,0
623,Mapping the state of the art of scientific pro...,0,0.328255,mapping the state of the art of scientific pro...,559.0,0,0,0
622,Validated Conceptual Model for Predictive Mapp...,0,0.343432,validated conceptual model for predictive mapp...,558.0,0,0,0


In [44]:
# total = 0
# # nan_rows = tmp_df.isna().any(axis=1)
# rows_with_nan = tmp_df[tmp_df.isna().any(axis=1)]

# # for i in nan_rows:
# #     if i:
# #         print(total, i)
# #         total+=1
# print(len(rows_with_nan))
# # print(rows_with_nan.loc[0])
# rows_with_nan.head()


98


,Titles,Was Selected?,SVM_proba,Titles_Formated,_match_index,Claes,Marcos,Katia
65,Researching Dynamic Phenomena in Complex Organ...,0,0.720192,researching dynamic phenomena in complex organ...,NaN,0,0,0
66,Researching Dynamic Phenomena in Complex Organ...,0,0.720192,researching dynamic phenomena in complex organ...,NaN,0,0,0
67,Researching Dynamic Phenomena in Complex Organ...,0,0.720192,researching dynamic phenomena in complex organ...,NaN,0,0,0
68,Researching Dynamic Phenomena in Complex Organ...,0,0.720192,researching dynamic phenomena in complex organ...,NaN,0,0,0
69,Researching Dynamic Phenomena in Complex Organ...,0,0.720192,researching dynamic phenomena in complex organ...,NaN,0,0,0


In [ ]:
# DEBUG TESTING
# To remove columns...
# tmp_df2.

# Problema: Remover titles que estão no CSV dos reviewers mas não no do ML... 
# OU simplesmente deletar na mão

In [ ]:
# merged_dataframe = ml_pd.merge(reviewers_pd, on='Titles', how='outer')
# print(len(merged_dataframe))
# merged_dataframe.head(8)

###### Pandas Functions to Combine CSVs (merge, join...)

In [10]:
# # Merge ML's and Reviewers' CSVs into one, by matching Titles

# # formated_reviewers_pd


# # tmp_df = pd.merge(formated_ml_df, formated_reviewers_df, on='_match_index', how='inner')
# tmp_df = pd.merge(formated_ml_df, formated_reviewers_df.drop(['Titles', 'Titles_Formated'], axis=1), on='_match_index', how='inner')

# # tmp_df['_match_index'].fillna(placeholder, inplace=True)
# tmp_df = tmp_df.dropna(subset=['_match_index'])


# print(len(tmp_df))
# tmp_df.head(10)

NameError: name 'formated_ml_df' is not defined

In [ ]:
# # Merge ML's and Reviewers' CSVs into one, by matching Titles
# merged_dataframe = pd.merge(ml_pd, reviewers_pd, on='Titles', how='outer')
# print(len(merged_dataframe))
# merged_dataframe.head(8)

# RETHINK OF THIS SOLUTION END

#



#

### Pandas Functions to Combine CSVs (merge, join...)

In [18]:
# # OBS: NEW CELL Sep27

# # TODO: Ideia ao invés de fazer merge pelo "Titles" e depois percorrer dois ponteiros comparando o str.lower()
# #       Já fazer o merge no ["titles"].lower() e depois percorrer dataset comparando "ML_titles" com "Reviewers_titles"

# # Convert both titles to lower case and then merge it
# ml_pd['Titles_Formated'] = ml_pd['Titles'].str.lower()
# reviewers_pd['Titles_Formated'] = reviewers_pd['Titles'].str.lower()

# merged_dataframe = pd.merge(ml_pd, reviewers_pd, on='Titles_Formated', how='outer')

# # Rename columns Titles_x and Titles_y
# merged_dataframe['Titles_ML'] = merged_dataframe['Titles_x'] 
# merged_dataframe['Titles_Reviewers'] = merged_dataframe['Titles_y'] 
# merged_dataframe = merged_dataframe.drop(['Titles_x', 'Titles_y'], axis=1)

# merged_dataframe_sorted = merged_dataframe.sort_values(by='Titles_Formated', ascending=True)
# merged_dataframe_sorted.head(20)

# print(len(merged_dataframe_sorted))
# merged_dataframe_sorted.head(8)


# # merged_dataframe_sorted = merged_dataframe_sorted[["Was Selected?	SVM_proba	Titles_Formated	Claes	Marcos	Katia	Titles_ML	Titles_Reviewers"]]
# # # Sorts the csv by title so that missmatch titles are next to each other
# # merged_dataframe_sorted = merged_dataframe.sort_values(by='Titles_Formated', ascending=True)


647


,Was Selected?,SVM_proba,Titles_Formated,Claes,Marcos,Katia,Titles_ML,Titles_Reviewers
153,0.0,0.970387,'let's not reinvent the wheel': a qualitative ...,0.0,0.0,0.0,'Let's not reinvent the wheel': A qualitative ...,'Let's not reinvent the wheel': A qualitative ...
6,0.0,0.653437,a bibliometric analysis of the turkish softwar...,0.0,0.0,0.0,A bibliometric analysis of the Turkish softwar...,A bibliometric analysis of the Turkish softwar...
506,1.0,0.997140,a case study of industry--academia communicati...,NaN,NaN,NaN,A case study of industry--academia communicati...,NaN
645,NaN,NaN,a case study of industry–academia communicatio...,2.0,2.0,2.0,NaN,A case study of industry–academia communicatio...
48,0.0,0.154115,a case study on artefact-based re improvement ...,0.0,0.0,0.0,A case study on artefact-based RE improvement ...,A case study on artefact-based RE improvement ...
316,0.0,0.170448,a chatbot for goal-oriented requirements modeling,0.0,0.0,0.0,A Chatbot for goal-oriented requirements modeling,A Chatbot for goal-oriented requirements modeling
378,0.0,0.609309,a colaboração universidade-empresa no contexto...,NaN,NaN,NaN,A COLABORAÇÃO UNIVERSIDADE-EMPRESA NO CONTEXTO...,NaN
602,NaN,NaN,a colaboração universidade-empresa no contexto...,0.0,0.0,0.0,NaN,A COLABORAÇÃO UNIVERSIDADE-EMPRESA NO CONTEXTO...


In [36]:
# Merge ML's and Reviewers' CSVs into one, by matching Titles
merged_dataframe = pd.merge(ml_pd, reviewers_pd, on='Titles', how='outer')
print(len(merged_dataframe))
merged_dataframe.head(8)

775


,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia
0,Operations Technology and Organizational Struc...,0.0,0.980527,NaN,NaN,NaN
1,Design science as nested problem solving,0.0,0.513517,NaN,NaN,NaN
2,A Large-Scale Empirical Study of Practitioners...,0.0,0.289893,NaN,NaN,NaN
3,Towards an Approach Matching CMD and DSR to Im...,0.0,0.983313,NaN,NaN,NaN
4,A PROPOSAL FOR USING DESIGN SCIENCE IN EDUCATI...,0.0,0.576047,0.0,0.0,0.0
5,University-industry collaboration and open sou...,0.0,0.567045,NaN,NaN,NaN
6,A bibliometric analysis of the Turkish softwar...,0.0,0.653437,0.0,0.0,0.0
7,An Improved Indoor Positioning Method Based on...,0.0,0.882294,NaN,NaN,NaN


In [37]:
# Add new column with formated titles to sort by it
merged_dataframe['Titles_Formated'] = merged_dataframe['Titles'].str.lower()

# Sorts the csv by title so that missmatch titles are next to each other
merged_dataframe_sorted = merged_dataframe.sort_values(by='Titles_Formated', ascending=True)
merged_dataframe_sorted.head(20)

,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated
152,'Let's not reinvent the wheel': A qualitative ...,0.0,0.970387,0.0,0.0,0.0,'let's not reinvent the wheel': a qualitative ...
6,A bibliometric analysis of the Turkish softwar...,0.0,0.653437,0.0,0.0,0.0,a bibliometric analysis of the turkish softwar...
504,A case study of industry--academia communicati...,1.0,0.997140,NaN,NaN,NaN,a case study of industry--academia communicati...
772,A case study of industry–academia communicatio...,NaN,NaN,2.0,2.0,2.0,a case study of industry–academia communicatio...
48,A case study on artefact-based RE improvement ...,0.0,0.154115,0.0,0.0,0.0,a case study on artefact-based re improvement ...
314,A Chatbot for goal-oriented requirements modeling,0.0,0.170448,0.0,0.0,0.0,a chatbot for goal-oriented requirements modeling
376,A COLABORAÇÃO UNIVERSIDADE-EMPRESA NO CONTEXTO...,0.0,0.609309,NaN,NaN,NaN,a colaboração universidade-empresa no contexto...
622,A COLABORAÇÃO UNIVERSIDADE-EMPRESA NO CONTEXTO...,NaN,NaN,0.0,0.0,0.0,a colaboração universidade-empresa no contexto...
535,A collaborative autoethnographic analysis of i...,0.0,1.000000,0.0,1.0,2.0,a collaborative autoethnographic analysis of i...
322,A Collaborative Method for Identification and ...,0.0,0.021194,0.0,0.0,0.0,a collaborative method for identification and ...


#### Separate the rows containing NaN values from the Titles that were equal in both dataframes

In [38]:
nan_rows = merged_dataframe_sorted.isna().any(axis=1)
missmatched_df = merged_dataframe[nan_rows]
missmatched_df = missmatched_df.sort_values(by='Titles_Formated', ascending=True)
missmatched_df.head()

/tmp/ipykernel_44107/3866861985.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  missmatched_df = merged_dataframe[nan_rows]


,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated
504,A case study of industry--academia communicati...,1.0,0.997140,NaN,NaN,NaN,a case study of industry--academia communicati...
772,A case study of industry–academia communicatio...,NaN,NaN,2.0,2.0,2.0,a case study of industry–academia communicatio...
376,A COLABORAÇÃO UNIVERSIDADE-EMPRESA NO CONTEXTO...,0.0,0.609309,NaN,NaN,NaN,a colaboração universidade-empresa no contexto...
622,A COLABORAÇÃO UNIVERSIDADE-EMPRESA NO CONTEXTO...,NaN,NaN,0.0,0.0,0.0,a colaboração universidade-empresa no contexto...
632,A critical view on PLM/ALM convergence in prac...,NaN,NaN,0.0,0.0,0.0,a critical view on plm/alm convergence in prac...


In [39]:
# Resets the index so you can loop throught it using from i=0 to N like an array
missmatched_df = missmatched_df.reset_index()
# missmatched_df = missmatched_df.drop(columns='level_0')
missmatched_df.head(20)


,index,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated
0,504,A case study of industry--academia communicati...,1.0,0.997140,NaN,NaN,NaN,a case study of industry--academia communicati...
1,772,A case study of industry–academia communicatio...,NaN,NaN,2.0,2.0,2.0,a case study of industry–academia communicatio...
2,376,A COLABORAÇÃO UNIVERSIDADE-EMPRESA NO CONTEXTO...,0.0,0.609309,NaN,NaN,NaN,a colaboração universidade-empresa no contexto...
3,622,A COLABORAÇÃO UNIVERSIDADE-EMPRESA NO CONTEXTO...,NaN,NaN,0.0,0.0,0.0,a colaboração universidade-empresa no contexto...
4,632,A critical view on PLM/ALM convergence in prac...,NaN,NaN,0.0,0.0,0.0,a critical view on plm/alm convergence in prac...
5,94,A Critical View on PLM/ALM Convergence in Prac...,0.0,0.880583,NaN,NaN,NaN,a critical view on plm/alm convergence in prac...
6,692,A dissection of the test-driven development pr...,NaN,NaN,0.0,0.0,0.0,a dissection of the test-driven development pr...
7,196,A Dissection of the Test-Driven Development Pr...,0.0,0.280250,NaN,NaN,NaN,a dissection of the test-driven development pr...
8,427,A framework to improve university--industry co...,1.0,0.918894,NaN,NaN,NaN,a framework to improve university--industry co...
9,2,A Large-Scale Empirical Study of Practitioners...,0.0,0.289893,NaN,NaN,NaN,a large-scale empirical study of practitioners...


### Compare strings

In [40]:
# pip install thefuzz
from thefuzz import fuzz, process
from copy import deepcopy

In [41]:
def merge_not_NaN_values(df: pd.DataFrame, index):
    # print(df.loc[index])
    if np.isnan(df.loc[index, 'SVM_proba']) and not(np.isnan(df.loc[index, 'Marcos'])):
        # print('Row 1 is Reviewers and Row 2 is ML')
        return True
    elif np.isnan(df.loc[index, 'Marcos']) and not(np.isnan(df.loc[index, 'SVM_proba'])):
        # print('Row 2 is Reviewers and Row 1 is ML')
        return False
    else:
        raise KeyError('ERROR on merge_not_NaN_values')

In [42]:
# matched_df = merged_dataframe.dropna(subset=merged_dataframe.columns[merged_dataframe.isna().any()].tolist())
i = 0
# similarity_treshold = 85  # 176 missing papers...
# similarity_treshold = 75  # 176 missing papers...
similarity_treshold = 50  # Reduced to 110 missing papers...

similar_matches_df = deepcopy(missmatched_df)
similar_matches_df['Similarity'] = np.nan  # Add new column and set all rows to be NaN

fuzz_match_count = 0
while(i < len(similar_matches_df)-1):
    # print(i)
    curr_row = similar_matches_df.loc[i]['Titles_Formated']
    next_row = similar_matches_df.loc[i+1]['Titles_Formated']

    # Attempt to find match by using the fuzz
    similarity = fuzz.partial_ratio(curr_row, next_row) 

    if similarity >= similarity_treshold: # Same title
        fuzz_match_count+=1
        if merge_not_NaN_values(similar_matches_df, i):
            # Copy values from curr_row into next_row for the second and third columns
            similar_matches_df.iloc[i+1, 4:7] = similar_matches_df.iloc[i, 4:7]

            # Copy values from next_row into curr_row for the Reviewers columns
            similar_matches_df.iloc[i, 2:4] = similar_matches_df.iloc[i+1, 2:4]

        else:
            # Copy values from next_row into curr_row for the Reviewers columns
            similar_matches_df.iloc[i, 4:7] = similar_matches_df.iloc[i+1, 4:7]

            # Copy values from curr_row into next_row for the second and third columns
            similar_matches_df.iloc[i+1, 2:4] = similar_matches_df.iloc[i, 2:4]


        # Copy index and title from curr_row to next_row
        similar_matches_df.iloc[i, 0:2] = similar_matches_df.iloc[i+1, 0:2]
        similar_matches_df.at[i, 'Similarity'] = similar_matches_df.at[i+1, 'Similarity'] = similarity
        i+=2

    else: # Not the same
        i+=1

print('Fuzz matches:', fuzz_match_count)
similar_matches_df.head()


Fuzz matches: 192


,index,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated,Similarity
0,772,A case study of industry–academia communicatio...,1.0,0.997140,2.0,2.0,2.0,a case study of industry--academia communicati...,98.0
1,772,A case study of industry–academia communicatio...,1.0,0.997140,2.0,2.0,2.0,a case study of industry–academia communicatio...,98.0
2,622,A COLABORAÇÃO UNIVERSIDADE-EMPRESA NO CONTEXTO...,0.0,0.609309,0.0,0.0,0.0,a colaboração universidade-empresa no contexto...,92.0
3,622,A COLABORAÇÃO UNIVERSIDADE-EMPRESA NO CONTEXTO...,0.0,0.609309,0.0,0.0,0.0,a colaboração universidade-empresa no contexto...,92.0
4,94,A Critical View on PLM/ALM Convergence in Prac...,0.0,0.880583,0.0,0.0,0.0,a critical view on plm/alm convergence in prac...,100.0


In [ ]:
# np.isnan(new_df.iloc[19, 2])
# new_df.iloc[1, 4:7]
# new_df.head()

In [43]:
similar_matches_df.head(20)

,index,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated,Similarity
0,772,A case study of industry–academia communicatio...,1.0,0.997140,2.0,2.0,2.0,a case study of industry--academia communicati...,98.0
1,772,A case study of industry–academia communicatio...,1.0,0.997140,2.0,2.0,2.0,a case study of industry–academia communicatio...,98.0
2,622,A COLABORAÇÃO UNIVERSIDADE-EMPRESA NO CONTEXTO...,0.0,0.609309,0.0,0.0,0.0,a colaboração universidade-empresa no contexto...,92.0
3,622,A COLABORAÇÃO UNIVERSIDADE-EMPRESA NO CONTEXTO...,0.0,0.609309,0.0,0.0,0.0,a colaboração universidade-empresa no contexto...,92.0
4,94,A Critical View on PLM/ALM Convergence in Prac...,0.0,0.880583,0.0,0.0,0.0,a critical view on plm/alm convergence in prac...,100.0
5,94,A Critical View on PLM/ALM Convergence in Prac...,0.0,0.880583,0.0,0.0,0.0,a critical view on plm/alm convergence in prac...,100.0
6,196,A Dissection of the Test-Driven Development Pr...,0.0,0.280250,0.0,0.0,0.0,a dissection of the test-driven development pr...,100.0
7,196,A Dissection of the Test-Driven Development Pr...,0.0,0.280250,0.0,0.0,0.0,a dissection of the test-driven development pr...,100.0
8,427,A framework to improve university--industry co...,1.0,0.918894,NaN,NaN,NaN,a framework to improve university--industry co...,NaN
9,679,A large-scale empirical study of practitioners...,0.0,0.289893,0.0,0.0,0.0,a large-scale empirical study of practitioners...,100.0


#### Combine new matches with other dataframe

In [44]:
# Get the first matches
original_matched_df = merged_dataframe_sorted.dropna(how='any')
# print(len(original_matched_df))
# original_matched_df.head()


# Get only the new rows that got a match
new_matches_df = similar_matches_df[(~similar_matches_df['Was Selected?'].isna()) & (~similar_matches_df['SVM_proba'].isna()) 
                                         & (~similar_matches_df['Claes'].isna()) & (~similar_matches_df['Marcos'].isna()) 
                                         & (~similar_matches_df['Katia'].isna())] 
# Drop duplicated rows from new matches (get df with unique Titles)
columns_to_check = ['index', 'Titles']
new_matches_df = new_matches_df[~new_matches_df.duplicated(subset=columns_to_check, keep='first')]
# print(len(new_matches_df))
# new_matches_df.head()              


In [45]:
print(len(original_matched_df))
original_matched_df.head()

350


,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated
152,'Let's not reinvent the wheel': A qualitative ...,0.0,0.970387,0.0,0.0,0.0,'let's not reinvent the wheel': a qualitative ...
6,A bibliometric analysis of the Turkish softwar...,0.0,0.653437,0.0,0.0,0.0,a bibliometric analysis of the turkish softwar...
48,A case study on artefact-based RE improvement ...,0.0,0.154115,0.0,0.0,0.0,a case study on artefact-based re improvement ...
314,A Chatbot for goal-oriented requirements modeling,0.0,0.170448,0.0,0.0,0.0,a chatbot for goal-oriented requirements modeling
535,A collaborative autoethnographic analysis of i...,0.0,1.000000,0.0,1.0,2.0,a collaborative autoethnographic analysis of i...


In [46]:
print(len(new_matches_df))
new_matches_df.head()

182


,index,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated,Similarity
0,772,A case study of industry–academia communicatio...,1.0,0.997140,2.0,2.0,2.0,a case study of industry--academia communicati...,98.0
2,622,A COLABORAÇÃO UNIVERSIDADE-EMPRESA NO CONTEXTO...,0.0,0.609309,0.0,0.0,0.0,a colaboração universidade-empresa no contexto...,92.0
4,94,A Critical View on PLM/ALM Convergence in Prac...,0.0,0.880583,0.0,0.0,0.0,a critical view on plm/alm convergence in prac...,100.0
6,196,A Dissection of the Test-Driven Development Pr...,0.0,0.280250,0.0,0.0,0.0,a dissection of the test-driven development pr...,100.0
9,679,A large-scale empirical study of practitioners...,0.0,0.289893,0.0,0.0,0.0,a large-scale empirical study of practitioners...,100.0


#### Create new DF combining all matches (this should be equal to ALL titles used in the testing set for ML)

In [47]:
# Merge dataframe with all results
columns_to_merge = ['Titles', 'Was Selected?', 'SVM_proba', 'Claes', 'Marcos', 'Katia']
merged_df = original_matched_df.merge(new_matches_df[columns_to_merge], on=columns_to_merge, how='outer')


In [48]:
# Remove the last column 'Titles_Formated' from final dataframe
merged_df = merged_df.drop(columns='Titles_Formated')
print(len(merged_df))
merged_df.head()

532


,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia
0,'Let's not reinvent the wheel': A qualitative ...,0.0,0.970387,0.0,0.0,0.0
1,A bibliometric analysis of the Turkish softwar...,0.0,0.653437,0.0,0.0,0.0
2,A case study on artefact-based RE improvement ...,0.0,0.154115,0.0,0.0,0.0
3,A Chatbot for goal-oriented requirements modeling,0.0,0.170448,0.0,0.0,0.0
4,A collaborative autoethnographic analysis of i...,0.0,1.000000,0.0,1.0,2.0


#### Verify DF with unmatched ML titles (this shouldn't happen, check for specific titles to know what went wrong)


In [49]:
# Filtered ML (results that still may be missing...)
not_found_ml_titles_df = similar_matches_df[(~similar_matches_df['Was Selected?'].isna()) & (~similar_matches_df['SVM_proba'].isna()) 
                                         & (similar_matches_df['Claes'].isna()) & (similar_matches_df['Marcos'].isna())]  
print(len(not_found_ml_titles_df))
not_found_ml_titles_df.head(20)

53


,index,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated,Similarity
8,427,A framework to improve university--industry co...,1.0,0.918894,NaN,NaN,NaN,a framework to improve university--industry co...,NaN
15,306,A Pearson Correlation Analysis of the Software...,0.0,0.242938,NaN,NaN,NaN,a pearson correlation analysis of the software...,NaN
20,158,A research framework for building SPI proposal...,1.0,0.188098,NaN,NaN,NaN,a research framework for building spi proposal...,NaN
21,156,A Researcher’s Experiences in Supporting Indus...,1.0,0.995268,NaN,NaN,NaN,a researcher’s experiences in supporting indus...,NaN
28,80,A Survey of Types of Industry-Academia Collabo...,0.0,1.000000,NaN,NaN,NaN,a survey of types of industry-academia collabo...,NaN
49,133,Agile Big Data Analytics for Web-Based Systems...,0.0,0.141111,NaN,NaN,NaN,agile big data analytics for web-based systems...,NaN
52,363,An Action Research for User Requirements Proce...,0.0,0.375654,NaN,NaN,NaN,an action research for user requirements proce...,NaN
55,73,An Exploratory Study on Technology Transfer in...,1.0,1.000000,NaN,NaN,NaN,an exploratory study on technology transfer in...,NaN
86,490,Bridging the ‘Valley of Death’: Can Agile Prin...,0.0,1.000000,NaN,NaN,NaN,bridging the industry-academia collaboration g...,59.0
87,490,Bridging the ‘Valley of Death’: Can Agile Prin...,0.0,1.000000,NaN,NaN,NaN,bridging the ‘valley of death’: can agile prin...,59.0


### Use difflib to search for each ML unmatched title the closest match on the REV df

In [50]:
# matched_df = merged_dataframe.dropna(subset=merged_dataframe.columns[merged_dataframe.isna().any()].tolist())
i = 0
# similarity_treshold = 85  
# similarity_treshold = 75  
# similarity_treshold = 50   # 567 matches (only missed 21... REMEMBER THAT ML CSV IS BIGGER THAN REVIEWERS'...)
similarity_treshold = 80   # 567 matches (only missed 21... REMEMBER THAT ML CSV IS BIGGER THAN REVIEWERS'...)
difflib_treshold = similarity_treshold/100


# Format both dataframes to lower case so is easier to match
formated_ml_df = deepcopy(not_found_ml_titles_df)
formated_ml_df['_match_index'] = np.nan  # Add new column and set all rows to be NaN
formated_ml_df = formated_ml_df.drop(['Similarity'], axis=1)
formated_ml_df = formated_ml_df.reset_index()

formated_reviewers_df = deepcopy(reviewers_pd)
formated_reviewers_df['Titles_Formated'] = formated_reviewers_df['Titles'].str.lower()
formated_reviewers_df['_match_index'] = np.nan  # Add new column and set all rows to be NaN

difflib_match_count = 0
matches = dict()

while(i < len(formated_ml_df)-1):
# while(i < 3):
    curr_ml_title = formated_ml_df.loc[i]['Titles_Formated']

    # Match by difflib
    difflib_match = difflib.get_close_matches(curr_ml_title, formated_reviewers_df['Titles_Formated'], n=1, cutoff=difflib_treshold)
   
    if difflib_match:
            # print('DIFF LIB:\n\t ml_title: {}\n\tmatch: {}'.format(curr_ml_title, difflib_match))
            difflib_match_count+=1

            # # Use boolean indexing to filter rows where 'Titles' column is equal to 'string_value'
            matched_row_index = formated_reviewers_df[formated_reviewers_df['Titles_Formated'] == difflib_match[0]].index[0]

            # Update the column '_match_index' on both dataframes to use to merge them later
            formated_reviewers_df.at[matched_row_index, '_match_index'] =  difflib_match_count
            formated_ml_df.at[i, '_match_index'] =  difflib_match_count


            if difflib_match_count not in matches:
                  matches[difflib_match_count] = { 'ml_title': curr_ml_title, 'reviewer_match': difflib_match[0] }
    i+=1

print('Difflib matches:', difflib_match_count)
# pd.DataFrame.from_dict(matches).head()


Difflib matches: 15


In [55]:
for c,index in enumerate(matches):
    if c < 7:
        print('ORIGINAL:   ', matches[index]['ml_title'])
        print('REMATCHE:   ', matches[index]['reviewer_match'])
        print('\n')


ORIGINAL:    a pearson correlation analysis of the softwareengineering practice in micro and small-sizedsoftware industry of sinaloa, mexico
REMATCHE:    a pearson correlation analysis of the software engineering practice in micro and small-sized software industry of sinaloa, mexico


ORIGINAL:    a survey of types of industry-academia collaboration
REMATCHE:    a survey of types of industry-academia collaboration


ORIGINAL:    bridging the industry-academia collaboration gap a focus towards final year projects
REMATCHE:    bridging the industry-academia collaboration gap a focus towards final year projects


ORIGINAL:    bridging the ‘valley of death’: can agile principles be applied in industry-academia research and innovation projects?
REMATCHE:    bridging the 'valley of death': can agile principles be applied in industry-academia research and innovation projects?


ORIGINAL:    case studies of industry-academia research collaborations for software development with agile
REMATCHE:

In [59]:
for c,index in enumerate(matches):
    if c >= 7 and c < 14:
        print('ORIGINAL:   ', matches[index]['ml_title'])
        print('REMATCHE:   ', matches[index]['reviewer_match'])
        print('\n')


ORIGINAL:    continuous management of design- and run-time artifacts for self-adaptive systems
REMATCHE:    continuous management of design-and run-time artifacts for self-adaptive systems


ORIGINAL:    guiding the selection of research methodology in industryâacademia collaboration in software engineering
REMATCHE:    guiding the selection of research methodology in industry–academia collaboration in software engineering


ORIGINAL:    industry--academia collaborations in software testing: experience and success stories from canada and turkey
REMATCHE:    industry–academia collaborations in software testing: experience and success stories from canada and turkey


ORIGINAL:    model-based generation of test scripts across product variants: an experience report from the railway industry
REMATCHE:    model‐based generation of test scripts across product variants: an experience report from the railway industry


ORIGINAL:    modeling the software prototyping process in a research conte

In [53]:
for c,index in enumerate(matches):
    if c >= 14:
        print('ORIGINAL:   ', matches[index]['ml_title'])
        print('REMATCHE:   ', matches[index]['reviewer_match'])
        print('\n')

ORIGINAL:    testdcat: catalog of test debt subtypes and management activities
REMATCHE:    testdcat 3.0: catalog of test debt subtypes and management activities




#### Get the sub dataframe of each one to merge the matches

In [60]:
# Reviewers dataframe
new_re_matches = formated_reviewers_df
new_re_matches = new_re_matches.dropna(subset=['_match_index'])
print('New matches count:', len(new_re_matches))
new_re_matches = new_re_matches.reset_index()
new_re_matches.head()

New matches count: 15


,index,Titles,Claes,Marcos,Katia,Titles_Formated,_match_index
0,7,Case studies of industry-academia research col...,0,0,0,case studies of industry-academia research col...,5.0
1,14,Guiding the selection of research methodology ...,0,0,0,guiding the selection of research methodology ...,9.0
2,41,A survey of types of industry-academia collabo...,2,0,2,a survey of types of industry-academia collabo...,2.0
3,58,A Pearson Correlation Analysis of the Software...,0,0,0,a pearson correlation analysis of the software...,1.0
4,124,Bridging the industry-academia collaboration g...,0,0,0,bridging the industry-academia collaboration g...,3.0


In [61]:
# ML dataframe
print('Old missmatch count:', len(formated_ml_df))
new_ml_matches = formated_ml_df
new_ml_matches = new_ml_matches.dropna(subset=['_match_index'])
print('New matches count:', len(new_ml_matches))
new_ml_matches = new_ml_matches.iloc[: ,2:]
new_ml_matches = new_ml_matches.reset_index()
new_ml_matches.head()

Old missmatch count: 53
New matches count: 15


,index,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated,_match_index
0,1,A Pearson Correlation Analysis of the Software...,0.0,0.242938,NaN,NaN,NaN,a pearson correlation analysis of the software...,1.0
1,4,A Survey of Types of Industry-Academia Collabo...,0.0,1.000000,NaN,NaN,NaN,a survey of types of industry-academia collabo...,2.0
2,8,Bridging the ‘Valley of Death’: Can Agile Prin...,0.0,1.000000,NaN,NaN,NaN,bridging the industry-academia collaboration g...,3.0
3,9,Bridging the ‘Valley of Death’: Can Agile Prin...,0.0,1.000000,NaN,NaN,NaN,bridging the ‘valley of death’: can agile prin...,4.0
4,10,Case Studies of Industry-Academia Research Col...,0.0,0.999997,NaN,NaN,NaN,case studies of industry-academia research col...,5.0


##### Merge the new dataframes to get the new matches

In [62]:
# First join the results
new_matches_df = pd.merge(new_ml_matches.drop(['Claes', 'Marcos', 'Katia', 'index'], axis=1), 
                          new_re_matches.drop(['Titles', 'Titles_Formated', 'index'], axis=1), on='_match_index', how='outer')

print(len(new_matches_df))
new_matches_df.head()

15


,Titles,Was Selected?,SVM_proba,Titles_Formated,_match_index,Claes,Marcos,Katia
0,A Pearson Correlation Analysis of the Software...,0.0,0.242938,a pearson correlation analysis of the software...,1.0,0,0,0
1,A Survey of Types of Industry-Academia Collabo...,0.0,1.000000,a survey of types of industry-academia collabo...,2.0,2,0,2
2,Bridging the ‘Valley of Death’: Can Agile Prin...,0.0,1.000000,bridging the industry-academia collaboration g...,3.0,0,0,0
3,Bridging the ‘Valley of Death’: Can Agile Prin...,0.0,1.000000,bridging the ‘valley of death’: can agile prin...,4.0,0,0,1
4,Case Studies of Industry-Academia Research Col...,0.0,0.999997,case studies of industry-academia research col...,5.0,0,0,0


##### Append the new matches to the rest of them

In [63]:
# Now append the new matches to the rest
columns_to_merge = ['Titles', 'Was Selected?', 'SVM_proba', 'Claes', 'Marcos', 'Katia']
merged_df = merged_df.merge(new_matches_df[columns_to_merge], on=columns_to_merge, how='outer')

print(len(merged_df))
merged_df.head()



547


,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia
0,'Let's not reinvent the wheel': A qualitative ...,0.0,0.970387,0.0,0.0,0.0
1,A bibliometric analysis of the Turkish softwar...,0.0,0.653437,0.0,0.0,0.0
2,A case study on artefact-based RE improvement ...,0.0,0.154115,0.0,0.0,0.0
3,A Chatbot for goal-oriented requirements modeling,0.0,0.170448,0.0,0.0,0.0
4,A collaborative autoethnographic analysis of i...,0.0,1.000000,0.0,1.0,2.0


### Save file 

In [64]:
RESULTS_PATH = '/home/celo/Desktop/zenodo-table-tests'
# csv_file_path = '/home/mcostalonga/new-home/thesis/open-repo-papers-zenodo/zenodo-final-result/output2.csv'
csv_file_path = os.path.join(RESULTS_PATH, 'output.csv')

# Convert the DataFrame to a CSV file
merged_df.to_csv(csv_file_path, index=False)  # Set index=False to omit the index column

print(f'DataFrame has been successfully saved to {csv_file_path}')

DataFrame has been successfully saved to /home/celo/Desktop/zenodo-table-tests/output.csv


#### Verify AGAIN DF with unmatched ML titles (this shouldn't happen, check for specific titles to know what went wrong)


In [65]:
# ML dataframe

# Filtered ML (results that still may be missing...)
not_found_ml_titles_df = formated_ml_df[(formated_ml_df['_match_index'].isna())]  
print(len(formated_ml_df))
print(len(not_found_ml_titles_df))
formated_ml_df.head(20)

# print('Old missmatch count:', len(formated_ml_df))
# new_ml_matches = formated_ml_df
# new_ml_matches = new_ml_matches.dropna(subset=['_match_index'])
# print('New matches count:', len(new_ml_matches))
# new_ml_matches = new_ml_matches.iloc[: ,2:]
# new_ml_matches = new_ml_matches.reset_index()
# new_ml_matches.head()

53
38


,level_0,index,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated,_match_index
0,8,427,A framework to improve university--industry co...,1.0,0.918894,NaN,NaN,NaN,a framework to improve university--industry co...,NaN
1,15,306,A Pearson Correlation Analysis of the Software...,0.0,0.242938,NaN,NaN,NaN,a pearson correlation analysis of the software...,1.0
2,20,158,A research framework for building SPI proposal...,1.0,0.188098,NaN,NaN,NaN,a research framework for building spi proposal...,NaN
3,21,156,A Researcher’s Experiences in Supporting Indus...,1.0,0.995268,NaN,NaN,NaN,a researcher’s experiences in supporting indus...,NaN
4,28,80,A Survey of Types of Industry-Academia Collabo...,0.0,1.000000,NaN,NaN,NaN,a survey of types of industry-academia collabo...,2.0
5,49,133,Agile Big Data Analytics for Web-Based Systems...,0.0,0.141111,NaN,NaN,NaN,agile big data analytics for web-based systems...,NaN
6,52,363,An Action Research for User Requirements Proce...,0.0,0.375654,NaN,NaN,NaN,an action research for user requirements proce...,NaN
7,55,73,An Exploratory Study on Technology Transfer in...,1.0,1.000000,NaN,NaN,NaN,an exploratory study on technology transfer in...,NaN
8,86,490,Bridging the ‘Valley of Death’: Can Agile Prin...,0.0,1.000000,NaN,NaN,NaN,bridging the industry-academia collaboration g...,3.0
9,87,490,Bridging the ‘Valley of Death’: Can Agile Prin...,0.0,1.000000,NaN,NaN,NaN,bridging the ‘valley of death’: can agile prin...,4.0


In [85]:
formated_ml_df.head()

,level_0,index,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated,_match_index
0,8,427,A framework to improve university--industry co...,1.0,0.918894,NaN,NaN,NaN,a framework to improve university--industry co...,NaN
1,15,306,A Pearson Correlation Analysis of the Software...,0.0,0.242938,NaN,NaN,NaN,a pearson correlation analysis of the software...,1.0
2,20,158,A research framework for building SPI proposal...,1.0,0.188098,NaN,NaN,NaN,a research framework for building spi proposal...,NaN
3,21,156,A Researcher’s Experiences in Supporting Indus...,1.0,0.995268,NaN,NaN,NaN,a researcher’s experiences in supporting indus...,NaN
4,28,80,A Survey of Types of Industry-Academia Collabo...,0.0,1.000000,NaN,NaN,NaN,a survey of types of industry-academia collabo...,2.0


In [ ]:
# # tmp_df = pd.merge(formated_ml_df, formated_reviewers_df, on='_match_index', how='inner')
# tmp_df = pd.merge(formated_ml_df, formated_reviewers_df.drop(['Titles', 'Titles_Formated'], axis=1), on='_match_index', how='inner')

# # tmp_df['_match_index'].fillna(placeholder, inplace=True)
# # tmp_df = tmp_df.dropna(subset=['_match_index'])
# tmp_df = tmp_df.sort_values(by='_match_index', ascending=False)


# # Merge ML's and Reviewers' CSVs into one, by matching Titles
# merged_dataframe = pd.merge(ml_pd, reviewers_pd, on='Titles', how='outer')
# print(len(merged_dataframe))
# merged_dataframe.head(8)

#### Verify DF with unmatched Reviewers' titles (this may happen, reviewers had more titles than our dataset)


In [26]:
# Filtered Reviewers (results that still may be missing...)
not_found_reviewers_titles_df = similar_matches_df[(similar_matches_df['Was Selected?'].isna()) & (similar_matches_df['SVM_proba'].isna()) 
                                         & (~similar_matches_df['Claes'].isna()) & (~similar_matches_df['Marcos'].isna())]  
print(len(not_found_reviewers_titles_df))
not_found_reviewers_titles_df.head(20)




8


,index,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated,Similarity
84,639,Bridging the industry-academia collaboration g...,NaN,NaN,0,0,0,bridging the gap: voices from industry and res...,51.0
85,639,Bridging the industry-academia collaboration g...,NaN,NaN,0,0,0,bridging the industry-academia collaboration g...,51.0
112,675,Continuous management of design-and run-time a...,NaN,NaN,0,0,0,continuous management of design-and run-time a...,NaN
249,644,Modeling the Software Prototyping Process in a...,NaN,NaN,2,0,0,modeling the software prototyping process in a...,NaN
252,677,Motivating the contributions,NaN,NaN,0,0,0,model‐based generation of test scripts across ...,50.0
253,677,Motivating the contributions,NaN,NaN,0,0,0,motivating the contributions,50.0
366,642,Technology Transfer–Requirements Engineering R...,NaN,NaN,0,Panel summary,Panel summary,technology transfer–requirements engineering r...,NaN
406,597,Uma Pesquisa-Ação no Contexto de Processos de ...,NaN,NaN,0,0,0,uma pesquisa-ação no contexto de processos de ...,NaN


In [ ]:
# After dropping duplicateds...
unique_not_found_reviewers_titles_df = \
    not_found_reviewers_titles_df[~not_found_reviewers_titles_df.duplicated(subset=columns_to_check, keep='first')]

print(len(unique_not_found_reviewers_titles_df))
unique_not_found_reviewers_titles_df.head(20)

51


,index,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated,Similarity
19,633,A critical view on PLM/ALM convergence in prac...,NaN,NaN,0,0,0,a critical view on plm/alm convergence in prac...,NaN
20,693,A dissection of the test-driven development pr...,NaN,NaN,0,0,0,a dissection of the test-driven development pr...,NaN
22,705,A practical guide to select quality indicators...,NaN,NaN,0,0,0,a method to assess and argue for practical sig...,57.0
27,683,A retrospective analysis of SAC requirements: ...,NaN,NaN,0,0,0,a retrospective analysis of sac requirements: ...,NaN
30,630,A study and comparison of industrial vs. acade...,NaN,NaN,0,0,0,a study and comparison of industrial vs. acade...,NaN
...,...,...,...,...,...,...,...,...,...
69,728,Applying options thinking to shape generativit...,NaN,NaN,0,0,0,applying options thinking to shape generativit...,NaN
86,745,CONTEXTUALIZING COMPETENCE AND LEARNING FOR IN...,NaN,NaN,0,0,0,contextualizing competence and learning for in...,NaN
108,676,Continuous management of design-and run-time a...,NaN,NaN,0,0,0,continuous management of design-and run-time a...,NaN
138,609,Empirical studies in question-answering system...,NaN,NaN,0,0,0,empirical studies in question-answering system...,NaN


Fazer comparação de strings row a row:
- Comparar i com i++
- Usar thefuzz

--------------------------------------------------------------------------------------------------------------------

Caso de excesso no ML_pd (papers que precisam ser encontrados no Reviewers_pd mas não foram)
- Pensar em fazer nova busca para encontrá-los
- Was Selected? | SVM_proba != NaN
- Reviewers == NaN

Caso de excesso no Reviewers_pd (papers que foram avaliados pelos revisores mas não foram usados no testing_set)
- Descartar
- Was Selected? | SVM_proba == NaN
- Reviewers != NaN

----------------------------------------------------------------------------------------------------------------------

- Fazer novo merge no pandas que tinha dado match nos títulos (matched_df), pode ser concatenação 
- Reordenar pelos títulos 

Ao final, conferir se todos os X papers do ML.csv estão nesse novo pandas (verificar len)

### TODO: Criar caso de CSV mais complexo e testar para ver se script está funcionando...
- Possíveis problemas: Ordenação (ao ordenar estamos assumindo que os missmatchs estarão sempre um do lado do outro, o que pode não ser verdade caso Um paper tenha um caractere diferente no começo, esteja em lower x upper case...)

### Talvez a solução seja percorrer TODAS as linhas do CSV de REVIEWERS para CADA entrada do CSV de ML....

### Uma solução para o Upper x Lower case seria normalizar tudo para upper ou lower (OBS: Aparentemente o sorted coloca A B C antes de a b c...)

### Testar com os dois CSVs de verdades...

### OBS: By analysing the others notebooks, we selected which xlsx files got the best results for each test case and then manually copied the path of the files here

In [ ]:
# matched_df = merged_dataframe.dropna(subset=merged_dataframe.columns[merged_dataframe.isna().any()].tolist())
i = 0
# similarity_treshold = 85  # 176 missing papers...
# similarity_treshold = 75  # 176 missing papers...
similarity_treshold = 50  # Reduced to 110 missing papers...

difflib_treshold = similarity_treshold/100

similar_matches_df = deepcopy(missmatched_df)
similar_matches_df['Similarity'] = np.nan  # Add new column and set all rows to be NaN

difflib_match_count = 0
fuzz_match_count = 0

while(i < len(similar_matches_df)-1):
    # print(i)
    curr_row = similar_matches_df.loc[i]['Titles']
    next_row = similar_matches_df.loc[i+1]['Titles']

    # Attempt to find match by using the fuzz
    similarity = fuzz.partial_ratio(curr_row, next_row) 
    if similarity > similarity_treshold:
        fuzz_match_count+=1

    # If the fuzz did not found a match try by difflib
    difflib_match = difflib.get_close_matches(curr_row, similar_matches_df['Titles'], n=1, cutoff=difflib_treshold)
    if difflib_match:
            # print('DIFF LIB:', difflib_match, similarity)
            difflib_match_count+=1

    # TODO: Can think of a loop starting from higher treshold to lower and attempting multiple matches...

    if similarity >= similarity_treshold: # TODO: or difflib_match: # Same title
        # if merge_not_NaN_values(similar_matches_df, i):
        #     # Copy values from curr_row into next_row for the second and third columns
        #     similar_matches_df.iloc[i+1, 4:7] = similar_matches_df.iloc[i, 4:7]

        #     # Copy values from next_row into curr_row for the Reviewers columns
        #     similar_matches_df.iloc[i, 2:4] = similar_matches_df.iloc[i+1, 2:4]

        # else:
        #     # Copy values from next_row into curr_row for the Reviewers columns
        #     similar_matches_df.iloc[i, 4:7] = similar_matches_df.iloc[i+1, 4:7]

        #     # Copy values from curr_row into next_row for the second and third columns
        #     similar_matches_df.iloc[i+1, 2:4] = similar_matches_df.iloc[i, 2:4]


        # # Copy index and title from curr_row to next_row
        # similar_matches_df.iloc[i, 0:2] = similar_matches_df.iloc[i+1, 0:2]
        # similar_matches_df.at[i, 'Similarity'] = similar_matches_df.at[i+1, 'Similarity'] = similarity
        i+=2

    else: # Not the same
        i+=1

print('Fuzz matches:', fuzz_match_count)
print('Difflib matches:', difflib_match_count)

similar_matches_df.head()


Fuzz matches: 155
Difflib matches: 265


,index,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated,Similarity
0,377,A COLABORAÇÃO UNIVERSIDADE-EMPRESA NO CONTEXTO...,0.0,0.609309,NaN,NaN,NaN,a colaboração universidade-empresa no contexto...,NaN
1,623,A COLABORAÇÃO UNIVERSIDADE-EMPRESA NO CONTEXTO...,NaN,NaN,0,0,0,a colaboração universidade-empresa no contexto...,NaN
2,94,A Critical View on PLM/ALM Convergence in Prac...,0.0,0.880583,NaN,NaN,NaN,a critical view on plm/alm convergence in prac...,NaN
3,197,A Dissection of the Test-Driven Development Pr...,0.0,0.280250,NaN,NaN,NaN,a dissection of the test-driven development pr...,NaN
4,488,A Method to Assess and Argue for Practical Sig...,0.0,0.999998,NaN,NaN,NaN,a method to assess and argue for practical sig...,NaN
